In [1]:
import pandas as pd

melb_data = pd.read_csv('file:///C:/Users/Marat/Desktop/Python/SkillFactory-python/Data/melb_data.csv', sep=',')

melb_data['Postcode'] = melb_data['Postcode'].astype('int64')
melb_data['Car'] = melb_data['Car'].astype('int64')
melb_data['Bedroom'] = melb_data['Bedroom'].astype('int64')
melb_data['Bathroom'] = melb_data['Bathroom'].astype('int64')
melb_data['Propertycount'] = melb_data['Propertycount'].astype('int64')
melb_data['YearBuilt'] = melb_data['YearBuilt'].astype('int64')

melb_df = melb_data.copy()

melb_df['Date'] = pd.to_datetime(melb_df['Date'])

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']

melb_df = melb_df.drop('YearBuilt', axis=1)

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek

# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
popular_stypes =street_types.value_counts().nlargest(10).index
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')

def get_street_type(weekday):
    if weekday in (5,6):
        return 1
    else:
        return 0
    
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_street_type)   

popular_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')

In [57]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца


In [58]:
melb_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          13580 non-null  int64         
 1   Suburb         13580 non-null  category      
 2   Address        13580 non-null  object        
 3   Rooms          13580 non-null  int64         
 4   Type           13580 non-null  category      
 5   Price          13580 non-null  float64       
 6   Method         13580 non-null  category      
 7   SellerG        13580 non-null  category      
 8   Date           13580 non-null  datetime64[ns]
 9   Distance       13580 non-null  float64       
 10  Postcode       13580 non-null  int64         
 11  Bedroom        13580 non-null  int64         
 12  Bathroom       13580 non-null  int64         
 13  Car            13580 non-null  int64         
 14  Landsize       13580 non-null  float64       
 15  BuildingArea   1358

In [51]:
melb_df['Suburb'].value_counts()

Reservoir         359
Richmond          260
Bentleigh East    249
Preston           239
Brunswick         222
                 ... 
Sandhurst           1
Bullengarook        1
Croydon South       1
Montrose            1
Monbulk             1
Name: Suburb, Length: 314, dtype: int64

In [52]:
popular_suburbs = melb_df['Suburb'].value_counts().nlargest(119).index


In [53]:
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburbs else 'other')

In [54]:
melb_df['Suburb'].value_counts()

other              2181
Reservoir           359
Richmond            260
Bentleigh East      249
Preston             239
                   ... 
Caulfield North      35
Chadstone            35
Hughesdale           35
Mont Albert          34
Alphington           34
Name: Suburb, Length: 120, dtype: int64

In [2]:
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

In [15]:
items_df.shape[0]

10

In [14]:
purchase_df.shape[0]

10

In [7]:
merged = items_df.merge(
    purchase_df,
    on = 'item_id'
    )

In [13]:
merged.shape[0]

9

In [16]:
merged['total'] = merged['price'] * merged['stock_count']

In [19]:
merged['total'].sum()

19729490